# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
#!pip install gmaps  
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json # testing
from pprint import pprint


# Import API key
from api_keys import g_key
#gmaps.configure(api_key=g_key)

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Import cities file as DataFrame
# Output File (CSV)
cities_pd = pd.read_csv("output_data/cities.csv")

cities_pd.head()

cities_pd.count()

City          578
Cloudiness    578
Country       572
Date          578
Humidity      578
Lat           578
Lng           578
Max Temp      578
Wind Speed    578
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_pd[["Lat", "Lng"]]

# Fill and convert to float
humidity = cities_pd["Humidity"].astype(float)


In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
#xcities_pd = cities_pd.dropna(inplace = True)

ideal_weather_df = pd.DataFrame(cities_pd)     #Create ideal dataframe, holds only rows that meets condition
ideal_weather_df.head()


#Any rows that doen's meet these conditions will be excluded/ dropped from dataframe
max_temp = (ideal_weather_df['Max Temp'] < 80) & (ideal_weather_df['Max Temp'] > 70)  # Condition 1: max temp between 71F to 79F  
wind_speed = ideal_weather_df['Wind Speed'] < 10                                      # Condition2:  Windsepp less than 10mph
zero_cloudiness = ideal_weather_df['Cloudiness'] == 0                                 # Condition 3: Cloudiness is 0

In [7]:
# Create ideal_weather_df meeting those three conditions  
ideal_weather_df[max_temp & wind_speed & zero_cloudiness]         

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
175,longyan,0,CN,1587023253,41,25.11,117.02,78.91,4.56
246,betafo,0,MG,1587023263,32,-19.83,46.85,74.59,8.63
248,quzhou,0,CN,1587023263,51,28.96,118.87,79.99,2.57
257,outjo,0,NaN,1587023264,35,-20.12,16.15,71.60,1.12
421,rehoboth,0,NaN,1587023289,12,-23.32,17.08,72.70,6.58
460,xiamen,0,CN,1587023027,50,24.48,118.08,78.80,8.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = (ideal_weather_df) # Create hotel_df
hotel_df["Hotel Name"]=""     # Add column "Hotel Name" to dataframe
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,longyearbyen,75,SJ,1587022396,64,78.22,15.64,37.4,13.87,
1,vaini,75,TO,1587022441,83,-21.20,-175.20,77.0,4.70,
2,kapaa,90,US,1587022665,83,22.08,-159.32,75.2,14.99,
3,ushuaia,63,AR,1587022624,75,-54.80,-68.30,46.4,3.36,
4,thompson,20,CA,1587022611,77,55.74,-97.86,6.8,6.93,


In [9]:
#set parameters for request url
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "location": f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}",
    "key": g_key
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response=requests.get(base_url, params=params)
response.url

# print response [200]...ok
pprint(response.json)


<bound method Response.json of <Response [200]>>


In [10]:
# Generate hotel list
   
hotel_name=[]

params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key
}
    

# Use the lat/lng to identify hotels
for index, row in cities_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    

    # convert to json
    name_address = name_address.json()
    print(index)
  

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_name.append(name_address["results"][0]["name"])
        
    except (IndexError): #(KeyError, IndexError)
        hotel_name.append("np.nan") # To handle missing data

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
# Add hotel_name to hotel_df['Hotel Name'] column
hotel_df['Hotel Name'] = hotel_name
hotel_df.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,longyearbyen,75,SJ,1587022396,64,78.22,15.64,37.4,13.87,"Radisson Blu Polar Hotel, Spitsbergen"
1,vaini,75,TO,1587022441,83,-21.20,-175.20,77.0,4.70,Paradise First Hotel
2,kapaa,90,US,1587022665,83,22.08,-159.32,75.2,14.99,Sheraton Kauai Resort at Coconut Beach
3,ushuaia,63,AR,1587022624,75,-54.80,-68.30,46.4,3.36,Los Cauquenes Resort & Spa
4,thompson,20,CA,1587022611,77,55.74,-97.86,6.8,6.93,Burntwood Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map


# Display Map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))